In [1]:
from math import ceil, floor
from openai import OpenAI
# from sagemaker.jumpstart.estimator import JumpStartEstimator
# from sagemaker.jumpstart.model import JumpStartModel
# from sagemaker.session import Session
# from sagemaker.s3 import S3Uploader
from tqdm import tqdm

# import boto3
import import_ipynb
import json
import os
import logging
import numpy as np
import pandas as pd
import random
# import sagemaker
import time

In [2]:
from utils import (
    brief_context_main,
    compare_answers,
    construct_synthetic_rankings,
    create_rag_dataset_with_synthetic_ranking,
    format_context_map_questions,
    format_context_reduce_prompt,
    format_context_reduce_prompt_gpt35,
    format_context_reduce_prompt_llama2,
    format_context_reduce_prompt_llama3,
    format_context_reduce_prompt_mixtral,
    format_options,
    format_prompt,
    format_prompt_gpt35,
    format_prompt_llama2,
    format_prompt_llama3,
    format_prompt_mixtral,
    get_bedrock_response_llama2,
    get_bedrock_response_llama3,
    get_bedrock_response_mixtral,
    get_bedrock_response_titan,
    get_model_response,
    get_model_response_bedrock,
    get_model_response_openai,
    sample_answer_positions,
    sample_question_ids,
    verify_answer,
)

In [3]:
# bedrock_runtime = boto3.client(
#     service_name = 'bedrock-runtime',
#     region_name = 'us-east-1',
# )

# openai_runtime = OpenAI(
#     api_key='sk-proj-RIQzg42aiuKfGmKARTDqT3BlbkFJslo7o3E8yNwkKkBFnY8F'
# )

bedrock_runtime = None

openai_runtime = OpenAI(
    api_key='OPENAI_API_KEY'
)

In [4]:
qid2query = json.load(
    open('./data/json_format/qid2query.json'))
qid2answer = json.load(
    open('./data/json_format/qid2answer.json'))
qid2retrieval = json.load(
    open('./data/json_format/qid2dense_retrieval_k64.json'))
qid2rank = json.load(
    open('./data/json_format/qid2sparse_rerank.json'))
qid2key_info = json.load(
    open('./data/json_format/qid2key_info.json'))
docs_content = json.load(
    open('./data/json_format/docs_content.json'))

In [5]:
model_id = 'gpt35-turbo-0126'

model_runtime = (
        openai_runtime if 'gpt' in model_id 
        else bedrock_runtime
    )

split = 2
top_k = 16
key_position = 8

dataset = json.load(open(
    './data/json_format/synthetic/dataset/pubmedqa/'
    f'LIM_k{top_k}_i{key_position}.json'))
fout = (
    './data/json_format/synthetic/results/pubmedqa/'
    f'cr_{model_id}_k{top_k}_i{key_position}.json'
)

bc_results = {}
for qid in tqdm(dataset):
    batch_size = top_k//split
    bc_results[qid] = brief_context_main(
        qid2query,
        qid2answer,
        qid2retrieval,
        qid,
        docs_content,
        batch_size, 
        top_k,
        model_id=model_id,
        model_runtime=model_runtime
    )
json.dump(bc_results, open(fout, 'w+'))
        

16 8


100%|█████████████████████████████████████████| 105/105 [06:24<00:00,  3.66s/it]
